# Making Do with Less: An Introduction to Comporessed Sensing

## Introduction

假設我們現在有7個硬幣在手上，裡面有一個硬幣是偽幣（重量比較重），我們知道正常的硬幣應該要多重，現在有一個精準的電子秤，最少測量幾次可以揪出那個偽幣是哪個？  
現在可以假設一個矩陣

每一行（column）代表一個硬幣有沒有被選到，1是有被選到，0是沒被選到，每一列（row）代表秤重的次數，這樣可以很快的得知哪個硬幣是偽幣，因為如果只有第一次秤的跟7枚正常硬幣的重量不一樣就知道是第一枚硬幣是偽幣，如果是第一次和第二次都跟正常重量不一樣就知道是第3枚是偽幣，其實就是把每枚硬幣用2進位表示，這樣如果我們有N枚硬幣只需要n = log2(N)的測量次數就可以得知哪個是偽幣。

現在我們把問題想大一些，N = 100，現在有100個硬幣，裡面有一小撮是偽幣，所以我們現在面臨的問題是有n個偽幣，n << N，在之前我們製造一個n\*N的0-1的矩陣$\Phi$，第k列（row）編碼成哪些硬幣要被挑中去秤，但因為我們不在假定只有一個硬幣是偽幣，所以原本的二進位編碼方式不能用了，那我們該怎麼製造我們的矩陣？

有很多人研究過這個問題，但在這我們要用一個特別的方式：用隨機來挑選被秤重的硬幣，b = $\Phi$x，我們的目標是從已知隨機選擇的$\Phi$和測量出的b推測出x的值，x也就是標示哪個硬幣是偽幣的vector。

一個列（row）比行（column）還少的矩陣，就會是個underdetermined system，如果consistent，就會有無限多組解，要解決這個狀況，最普遍的方式用regularize也是加些條件在x上，讓有一個唯一解x = x\*存在，這時困難點就在於要怎麼找到這個條件，讓x\*是我們想要的答案，在此註明我們要的解是sparse的，因為我們前提有說只有一小撮是偽幣。

一個普遍regularizing一個underdetermined linear system的方法是用Euclidean $\mathscr{l}^2$ norm，這可以用多變量微分解，但很不幸的，這裡不太適合這種regularization的方式：vector x\*如果用Euclidean $\mathscr{l}^2$ norm會有很多非零數，跟我們要的sparse解不一致。

因為我們相信真正的解是sparse的，所以蠻合理的我們要找的解是最少可能非零元素的x，我們定義$\Vert{x}\Vert_{0}$為在x裡有多少非零個數，然後問題就變成找一個$\Phi$x = b的解然後最小$\Vert{x}\Vert_{0}$，雖然可以把問題簡化了，但找最少非零計算上不太可行如果這個system太大時。

我們需要的是一個夠sparse的解但同時又要在可負擔的計算量內，$\mathscr{l}^1$ norm就是我們要找的東西，所以現在問題變成一個標轉的線性規劃問題，在$\Phi$x = b的限制下，最小化$\Vert{x}\Vert_{1}$，由於$\mid x_{i} \mid$不可微，所以就是一個convex optimization問題。

### Overview of Compressed Sensing

為什麼要先提到這個coin問題，因為跟compressed sensing很多地方很相似。現在是IoT和巨量資料的時代，一個sensor如果能收到越多資料越好，但就會有網路傳輸量問題，所以最好的方式就是壓縮資料等到傳到了再復原，但舊有的訊號處理是用uniform sampling的方式，我們希望有別的技術能突破現有的壓縮限制。

假如把原始要傳輸的資料想成用一個x vector表示，如果我們能找到一個很棒的扁扁的$\Phi$矩陣，使得$\Phi$x出來的b是一個長度比x小非常多的vector，然後傳輸的雙方其實都知道$\Phi$，這樣輸出方只要做一個矩陣相乘丟出b給接收方，接收方只要用$\Phi$和b還原x，這樣就從原本要傳輸x這麼多的資料變成只要傳輸b的資料量，非常美好，但這時就會有幾個問題出現了：

1. $\Phi$怎麼做出來？
2. 怎麼能確保$\Phi$和b還原的x是唯一解？

接下來就要解決這些問題

### The General Setting

在現實世界中x可能不是sparse的，那我們就把它轉換成sparse，方法是找出x = $\Psi$s，這個$\Psi$是個n x n的orthogonal matrix，s是個sparse，意思也就是x有可能在另一個$\mathbb{R}^n$空間的一個basis中有sparse representation，spanned by the columns of $\Psi$，如果是這樣的話就變成$\Psi\Phi$s = b，就要先解出s再解出x，

這段看不懂

接下來就把問題定義好：
我們要解一個linear system of equations

$\Phi$x = b

$\Phi$是一個n x N的sensing矩陣，n小於N非常多，我們先假定這個linear system是consistent（有解：一組或無限多組），但因為有可能有無限多組解（我們不想要傳輸過去解壓縮後發現居然有好幾種版本的資料，這樣會很麻煩），我們希望他是唯一解，所以我們要加些條件在上面，我們假定x是個"k-sparse"的vector，k代表x中至多不為0的數，k遠小於n，我們可以在這把這種vector的集合表示成$\Sigma_{k}$，那接下來又有幾個問題出現了：

1. 加入x是個k-sparse的vector這個條件真的會讓解是唯一解？如果是的話k, n, N的關係會是什麼？什麼狀況下的$\Phi$才會是有效的？
2. 在什麼狀況下解線性規劃$\Phi$x = b最小化$\mathscr{l}^1$ norm會成功獲得sparse的解？為什麼這樣做會成功？

Section 2會用檢驗null space of $\Phi$來確保k-sparse的解是唯一解，也就會是"restricted isometry property"，RIP這個性質會在section 3中解釋得更清楚，section 4會證明$\mathscr{l}^1$ norm可以成功獲得k-sparse的解

In [ ]:
## 